In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：使用导出数据集进行在线预测的自定义训练文本分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_text_classification_online_exported_ds.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_text_classification_online_exported_ds.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用Python的Vertex客户端库来训练和部署自定义文本分类模型进行在线预测，使用导出的`Dataset`资源。

### 数据集

本教程使用的数据集是来自[Kaggle Datasets](https://www.kaggle.com/ritresearch/happydb)的[Happy Moments dataset](https://www.kaggle.com/ritresearch/happydb)。您在本教程中将使用的数据集版本存储在公共云存储桶中。

### 目标

在本教程中，您将学习如何使用在Docker容器中使用Vertex客户端库从Python脚本中导出的`Dataset`资源创建自定义模型，然后对部署的模型进行预测。您还可以选择使用`gcloud`命令行工具或通过Google Cloud控制台在线创建模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 导出`Dataset`资源的清单。
- 为训练模型创建一个Vertex自定义作业。
- 导入导出的数据集清单。
- 训练模型。
- 检索并加载模型工件。
- 查看模型评估。
- 将模型上传为Vertex `Model`资源。
- 将`Model`资源部署到一个服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

成本

本教程使用Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您已安装了Vertex客户端库和Google *云存储*，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果可选的话，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您会获得$300的免费信用用于您的计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您也可以更改 `REGION` 变量，该变量用于整个笔记本的操作。下面是 Vertex 支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶来训练 Vertex。并非所有区域都支持所有 Vertex 服务。有关每个区域最新支持的信息，请参阅 [Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在现场教程会话中，您可能会使用一个共享的测试帐户或项目。为了避免在创建的资源上用户之间的名称冲突，您为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源的名称。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的Google Cloud帐号

**如果您正在使用Google Cloud Notebook**，您的环境已经经过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并在提示时按照说明进行身份验证，通过oAuth。

**否则**，请按照以下步骤进行操作：

在Cloud控制台中，转到[Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入名称，点击**创建**。

在**授予此服务帐号对项目的访问权限**部分，点击角色下拉列表。在筛选框中键入"Vertex"，并选择**Vertex管理员**。在筛选框中键入"Storage Object Admin"，并选择**存储对象管理员**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务帐号密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，都需要执行以下步骤。**

当您使用 Vertex 客户端库提交一个自定义训练作业时，您需要将包含训练代码的 Python 包上传到一个云存储桶中。Vertex 会从这个包中运行代码。在本教程中，Vertex 还会将训练作业产生的训练模型保存在同一个存储桶中。然后，您可以根据这个输出创建一个 `Endpoint` 资源，以便提供在线预测。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在的情况下才运行以下单元格创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查存储桶的内容来验证对您的云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设定变量

接下来，设定一些在教程中使用的变量。
### 导入库并定义常量

#### 导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点 API 服务的数据集、模型、作业、流水线和端点服务终端点。
- `PARENT`：数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

In [ ]:
# Text Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/text_1.0.0.yaml"
# Text Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/text_classification_single_label_io_format_1.0.0.yaml"

#### 硬件加速器

设置硬件加速器（例如，GPU）用于训练和预测。

设置变量`TRAIN_GPU / TRAIN_NGPU`和`DEPLOY_GPU / DEPLOY_NGPU`来使用支持 GPU 的容器映像以及分配给虚拟机实例的 GPU 数量。例如，要在每个 VM 上分配 4 个 Nvidia Telsa K80 GPU 使用 GPU 容器映像，您可以指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`来使用一个在 CPU 上运行的容器映像。

*注意*：TF 2.3 之前的 GPU 支持的版本将在本教程中加载自定义模型时失败。这是一个已知问题，在 TF 2.3 中已修复 —— 这是由生成在服务函数中的静态图操作引起的。如果您在自己的自定义模型上遇到此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）映像

接下来，我们将设置用于训练和预测的Docker容器映像

  - TensorFlow 1.15
    - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
    - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
  - TensorFlow 2.1
    - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
    - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
  - TensorFlow 2.2
    - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
    - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
  - TensorFlow 2.3
    - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
    - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
  - TensorFlow 2.4
    - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
    - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
  - XGBoost
    - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
  - Scikit-learn
    - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
  - Pytorch
    - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
    - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
    - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
    - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers).

  - TensorFlow 1.15
    - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
  - TensorFlow 2.1
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
  - TensorFlow 2.2
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
  - TensorFlow 2.3
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
  - XGBoost
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
  - Scikit-learn
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB的内存。
     - `n1-highmem`: 每个vCPU 6.5GB的内存。
     - `n1-highcpu`: 每个vCPU 0.9GB的内存。
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96\]个CPU核心

*注意：以下内容不支持训练:*

 - 标准`: 2个vCPUs
 - 高CPU`: 2, 4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

#教程

现在您已经准备好开始为Happy Moments创建自己的定制模型并训练。

## 设置客户端

Vertex客户端库采用客户端/服务器模型。在您的一侧（Python脚本），您将创建一个客户端，用于向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

-  Dataset服务用于`Dataset`资源。
-  Model服务用于`Model`资源。
-  Endpoint服务用于部署。
-  Job服务用于批量作业和自定义培训。
-  Prediction服务用于提供。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

数据集

现在你的客户准备好了，训练模型的第一步是创建一个托管的数据集实例，然后将标记数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。这个函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个Vertex `Dataset`资源（`aip.Dataset`），具有以下参数：
 - `display_name`：您选择的易读名称。
 - `metadata_schema_uri`：数据集类型的架构。
3. 调用客户端数据集服务方法`create_dataset`，具有以下参数：
 - `parent`：您的`数据库`、`模型`和`端点`资源的Vertex位置根路径。
 - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

一个`operation`对象是Vertex处理长时间运行操作的异步调用的方式。虽然这一步通常很快，但当您在项目中首次使用时，由于需要进行资源分配，可能会有较长的延迟。

您可以使用`operation`对象来获取操作的状态（例如，创建`Dataset`资源）或取消操作，通过调用一个操作方法：

| 方法       | 描述         |
| ----------- | ----------- |
| result()    | 等待操作完成并以JSON格式返回结果对象。      |
| running()   | 返回操作是否仍在运行的True/False。        |
| done()      | 返回操作是否已完成的True/False。 |
| canceled()  | 返回操作是否已取消的True/False。 |
| cancel()    | 取消操作（可能需要最多30秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("happydb-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

Vertex 的 `Dataset` 资源对您的文本数据有一些要求。

- 文本示例必须存储在 CSV 或 JSONL 文件中。

CSV文件

对于文本分类，CSV文件有一些要求：

- 无标题。
- 第一列是文本示例或文本文件的Cloud Storage路径（.txt后缀）。
- 第二列是标签。

云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为Cloud Storage中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

快速查看您的数据

您将使用存储在公共云存储桶中的Happy Moments数据集的一个版本，使用一个CSV索引文件。

从快速查看数据开始。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的 Vertex 数据集资源中。使用这个辅助函数 `import_data` 来导入数据。该函数执行以下操作：

- 使用 `Dataset` 客户端。
- 调用客户端方法 `import_data`，传入以下参数：
  - `name`: 您给 `Dataset` 资源的人类可读名称（例如，happydb）。
  - `import_configs`: 导入配置。

- `import_configs`: 一个包含字典的 Python 列表，具有键/值条目：
  - `gcs_sources`: 一个包含一个或多个索引文件路径的 URI 列表。
  - `import_schema_uri`: 标识标签类型的架构。

`import_data()` 方法返回一个长时间运行的 `operation` 对象。这将需要几分钟时间才能完成。如果您在进行实时教程，这将是一个提问问题或休息的好时机。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

### 导出数据集索引

接下来，您将把数据集索引导出到一个JSONL文件中，然后您的自定义训练作业将使用这个文件来获取数据和相应的标签，用于训练您的Happy Moments模型。使用这个辅助函数`export_data`来导出数据集索引。该函数执行以下操作：

- 使用数据集客户端。
- 调用客户端方法`export_data`，并传入以下参数：
 - `name`：您给数据集的人类可读名称（例如，happydb）。
 - `export_config`：导出配置。
- `export_config`：一个包含字典的Python列表，其中包含以下键值对：
 - `gcs_destination`：写入JSONL数据集索引文件的Cloud Storage存储桶。

`export_data()` 方法会返回一个长时间运行的`operation`对象。这将需要几分钟的时间才能完成。当导出完成时，辅助函数将返回长时间运行的操作和操作结果。

In [ ]:
EXPORT_FILE = BUCKET_NAME + "/export"


def export_data(dataset_id, gcs_dest):
    config = {"gcs_destination": {"output_uri_prefix": gcs_dest}}
    start_time = time.time()
    try:
        operation = clients["dataset"].export_data(
            name=dataset_id, export_config=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation, result
    except Exception as e:
        print("exception:", e)
        return None, None


_, result = export_data(dataset_id, EXPORT_FILE)

让我们快速查看导出的数据集索引文件的内容。当`export_data()`完成时，可以通过长时间运行操作的`result()`方法获得响应对象。响应对象包含以下属性：

- `exported_files`：导出的数据集索引文件的路径列表，在这种情况下将是一个文件。

您将获得导出的数据集索引文件的路径（`result.exported_files[0]`），然后显示文件中的前十个JSON对象，即数据项。

每个数据项的JSONL格式为：

    { "textGcsUri": path_to_the_text_file, "classificationAnnotation": { "displayName": label } }

In [ ]:
jsonl_index = result.exported_files[0]

! gsutil cat $jsonl_index | head

## 下架 `Model` 资源

现在从提供服务的 `Endpoint` 资源中下架您的 `Model` 资源。使用这个辅助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：在将 `Model` 资源部署到时由端点服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 Vertex 完全限定标识符所在的 `Endpoint` 资源。

这个函数调用端点客户端服务的 `undeploy_model` 方法，带有以下参数：

- `deployed_model_id`：在将 `Model` 资源部署时由端点服务返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 Vertex 完全限定标识符所在的 `Endpoint` 资源。
- `traffic_split`：如何在 `Endpoint` 资源上的其余部署模型之间拆分流量。

由于这是 `Endpoint` 资源上唯一部署的模型，您可以通过将其设置为空 {} 来简单地将 `traffic_split` 留空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理工作

要清理此项目中使用的所有GCP资源，您可以[删除用于本教程的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud Storage桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME